# TP4 Hopfield

In [ ]:
from asciiHandler import asciiStringToLettersMatrix
from hopfield import propagatePattern,calculateWeights,sgn,addNoise,printLetter,printLetterBipolar,printLetterBipolarMatrix,plotLetterMap,plotEnergy,isPattern,plotSpuriousStatesCountPerNoiseProbability
import numpy as np
import seaborn as sns
import random
from string import ascii_lowercase
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#PROPIEDADES DE CONFIGURACION
hopfieldProperties = {
    'seed': 10,
    'noiseProbability':0,
    'lettersCombo': ['o','g','c','d'],
}

In [ ]:
#SETEO DEL RANDOM SEED
np.random.seed(hopfieldProperties.get('seed'))
random.seed(hopfieldProperties.get('seed'))

In [ ]:
alphabet="""
 ###  ####   ###  ####  ##### #####  ###  #   # ##### ##### #   # #     #   #
#   # #   # #   # #   # #     #     #     #   #   #     #   #  #  #     ## ##
##### ####  #     #   # ####  ####  #  ## #####   #     #   ###   #     # # #
#   # #   # #   # #   # #     #     #   # #   #   #   # #   #  #  #     #   #
#   # ####   ###  ####  ##### #      ###  #   # ##### ###   #   # ##### #   #

#   #  ###  ####   ###  ####   ###  ##### #   # #   # #   # #   # #   # #####
##  # #   # #   # #   # #   # #       #   #   # #   # #   #  # #   # #     # 
# # # #   # ####  #   # ####   ###    #   #   #  # #  # # #   #     #     #  
#  ## #   # #     #  #  #   #     #   #   #   #  # #  ## ##  # #    #    #   
#   #  ###  #      ## # #   #  ###    #    ###    #   #   # #   #   #   #####"""

In [ ]:
#MOVE EACH ASCII TO A LETTER MATRIX
(alphabetASCII,alphabetASCIIBipolar)=asciiStringToLettersMatrix()

In [ ]:
#BUSCAR LETRAS MAS ORTOGONALES
ortogonality=np.zeros((len(alphabetASCIIBipolar),len(alphabetASCIIBipolar)))
alphabet = [c for c in ascii_lowercase]
print(np.dot(alphabetASCIIBipolar[1],alphabetASCIIBipolar[0]))
for i in range(0,len(alphabetASCIIBipolar)):
    for j in range(0,len(alphabetASCIIBipolar)):
        ortogonality[i][j]=np.dot(alphabetASCIIBipolar[i],alphabetASCIIBipolar[j])
df_cm = pd.DataFrame(ortogonality, index = [i for i in ascii_lowercase],
                  columns = [i for i in ascii_lowercase])
plt.figure(figsize = (15,9))
sns.heatmap(data=df_cm,cmap=sns.color_palette("vlag", as_cmap=True),annot=True)

#sns.heatmap(data=df_cm,cmap=sns.diverging_palette(250, 30, l=65, center="dark", as_cmap=True))
plt.title("Producto interno entre Letras")

In [ ]:
# CALCULAR 4 LETRAS MAS ORTOGONALES yMENOS ORTOGONALES
min_ortogonality_letters=[]
min_ortogonality=[25**6]
for i in range(0,len(alphabet)):
    for j in range(0,len(alphabet)):
        for k in range(0,len(alphabet)):
            for r in range(0,len(alphabet)):
                tot_ortogonality=ortogonality[i][j]*ortogonality[i][k]*ortogonality[i][r]*ortogonality[j][k]*ortogonality[j][r]*ortogonality[r][k]
                if(np.abs(tot_ortogonality)<=min_ortogonality[0] and i!=j and i!=k and i!=r and j!=k and j!=r and k!=r):
                    min_ortogonality_letters.insert(0,f"{ascii_lowercase[i]}{ascii_lowercase[j]}{ascii_lowercase[k]}{ascii_lowercase[r]}")
                    min_ortogonality.insert(0,np.abs(tot_ortogonality))
print(min_ortogonality_letters)
print(min_ortogonality)

In [ ]:
# CALCULAR 4 LETRAS MENOS ORTOGONALES
max_ortogonality_letters=[]
max_ortogonality=[0]
for i in range(0,len(alphabet)):
    for j in range(0,len(alphabet)):
        for k in range(0,len(alphabet)):
            for r in range(0,len(alphabet)):
                tot_ortogonality=ortogonality[i][j]*ortogonality[i][k]*ortogonality[i][r]*ortogonality[j][k]*ortogonality[j][r]*ortogonality[r][k]
                if(np.abs(tot_ortogonality)>=max_ortogonality[0] and i!=j and i!=k and i!=r and j!=k and j!=r and k!=r):
                    max_ortogonality_letters.insert(0,f"{ascii_lowercase[i]}{ascii_lowercase[j]}{ascii_lowercase[k]}{ascii_lowercase[r]}")
                    max_ortogonality.insert(0,np.abs(tot_ortogonality))
print(max_ortogonality_letters)
print(max_ortogonality)

In [ ]:
# AGARRAR 4 LETRAS
patterns_letters=hopfieldProperties['lettersCombo']
patterns=[]
for i in range(0,len(patterns_letters)):
    patterns.append(alphabetASCIIBipolar[ascii_lowercase.index(patterns_letters[i])])
# CALCULAR LOS PESOS DE LA RED
W=calculateWeights(patterns)



In [ ]:
#VERIFICAR PATRONES
results=[]
energies=[]
iterations=[]
noise_probability=hopfieldProperties['noiseProbability']
patternsWNoise=[]
# Add Noise
for i in range(0,len(patterns)):
    patternsWNoise.append(addNoise(noise_probability,patterns[i]))
# Propagate patterns with noise
for p in range(0,len(patterns_letters)):
    results.append([])
    energies.append([])
    results[p]= propagatePattern(W,patternsWNoise[p])[0]
    energies[p]= propagatePattern(W,patternsWNoise[p])[1]
    iterations.append(len(results[p])-1)

# Plot Results (1st Row => Original pattern
#               2nd Row => Pattern with noise
#               3rd+ Row => Propagation     )    
figure, axes = plt.subplots(max(iterations)+2, 4, sharex=True, figsize=(19,7))

for i in range(0,len(patterns)):
    plotLetterMap(patterns[i],axes=axes[0, i])
    plotLetterMap(patternsWNoise[i],axes=axes[1, i])
    for j in range(0,len(results[i])-1):
        plotLetterMap(results[i][j],axes=axes[j+2, i])





In [ ]:
#GRAFICAR LA ENERGIA EN FUNCION DE LAS ITERACIONES PARA LOS PATRONES CON RUIDO
plotEnergy(energies,patterns_letters)

In [ ]:
#GRAFICAR CANTIDAD DE ESTADOS ESPUREOS EN FUNCION DE LA PROBABILIDAD DE RUIDO UTILIZADA

#Crear patrones de letras
patterns_length=4
patterns_letters=[['k','n','s','v'],['a','u','k','j'],['s','p','a','q'],['a','b','c','d'],['v','t','r','q'],['o','g','c','d']]
patterns=[]
for i in range(0,len(patterns_letters)):
    patterns.append([])
    for j in range(0,len(patterns_letters[i])):
        patterns[i].append(alphabetASCIIBipolar[ascii_lowercase.index(patterns_letters[i][j])])

#Creamos una lista con las probabilidades de ruido a evaluar, y otra para ir guardando por cada una la cant de estados espureos
noiseProbabilities = [0,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
spuriousStatesCount=[]

#Por cada una, ejecutamos Hopfield y vemos cuantos estados espureos se obtienen
for p in range(0,len(patterns)):
    spuriousStatesCount.append([])
    for i in range(0,len(noiseProbabilities)):
        results=[]
        patternsWNoise=[]
        # Add Noise
        for j in range(0,100*patterns_length):
            patternsWNoise.append(addNoise(noiseProbabilities[i],patterns[p][j%patterns_length]))
        # Propagate patterns with noise
        spuriousStatesCountPerNoiseProbability = 0
        for k in range(0,len(patternsWNoise)):
            results.append([])
            W=calculateWeights(patterns[p])
            results[k]= propagatePattern(W,patternsWNoise[k])[0]
            if(not isPattern(results[k][-1],patterns[p])):
                spuriousStatesCountPerNoiseProbability+=1
        spuriousStatesCount[p].append(spuriousStatesCountPerNoiseProbability/400)
print(spuriousStatesCount)
#Finalmente, realizamos el grafico
plotSpuriousStatesCountPerNoiseProbability(patterns_letters,noiseProbabilities,spuriousStatesCount)